In [2]:
## psycopg was used to read from a local postgres db
# import psycopg2
import pandas as pd

# Stop and Frisk as a demograhic of New York

In [5]:
## for the purpose of contanising this project, the outputs of these queries
## were extracted and stored in the data folder 


# conn = psycopg2.connect(host="localhost",database="blm")

# # dataframe for correlating borough and population with sqf stops by borough and race
# sql = '''
# select t1.borough, t2.race, t1.est_pop, t2.sqf_count from (
# 	select lower(borough) as borough,
# 		case 
# 	 		when race = 'asian' then 'asian / pacific islander'
# 		 	when race = 'pacific' then 'asian / pacific islander'
# 			when race = 'other' then 'other'
# 			when race = 'two' then 'other'
# 			when race = 'native' then 'other'
# 			else lower(race)
# 		end as race_n, 
# 		sum (est_pop) as est_pop 
# 	from us.demo_boro db 
# 	where not race = 'population'
# 	group by borough, race_n
# 	order by borough, race_n
# ) t1
# left join (
# 	select lower(stop_location_boro_name) as borough,
# 		case 
# 			when suspect_race_description = 'BLACK HISPANIC' then 'hispanic_latino'
# 			when suspect_race_description = 'WHITE HISPANIC' then 'hispanic_latino'
# 			when suspect_race_description = 'AMER IND' then 'other'
# 			when suspect_race_description = 'AMERICAN INDIAN/ALASKAN N' then 'other'
# 			when suspect_race_description = 'ASIAN/PAC.ISL' then 'asian / pacific islander'
# 			when suspect_race_description = '' then 'other'
# 			else lower(suspect_race_description) 
# 		end as race,
# 		count(*) as sqf_count
# 	from us.sqf s 
# 	where stop_frisk_date > '2018-12-31'
# 		and not stop_location_boro_name = ''
# 	group by stop_location_boro_name, race
# 	order by stop_location_boro_name, race 
# ) t2
# on t1.borough = t2.borough
# 	and t1.race_n = t2.race
# order by t1.borough, t2.race 
# '''

# df1 = pd.read_sql_query(sql, conn)


# #creating new dataframe to store population information
# columns = ['borough','pop','sqf_total']
# df2 = pd.DataFrame(columns=columns)

# boroughs = ['Bronx','Brooklyn','Staten Island','Queens','Manhattan']
# x = 0
# df2['borough'] = boroughs

# for borough in boroughs:
#     df2['pop'].iloc[x] = pd.read_sql_query('''
#                             select est_pop from us.demo_boro db where race = 'population' and borough = '{}'
#                             '''.format(borough)
#                            ,conn)['est_pop'].iloc[0]
#     df2['sqf_total'].iloc[x] = pd.read_sql_query('''
#                                 select count (*) from us.sqf s 
#                                     where initcap(stop_location_boro_name) = '{}'
#                                     and stop_frisk_date >'2018-12-31'
#                                 '''.format(borough)
#                                ,conn)['count'].iloc[0]
#     x = x + 1 


In [3]:
df1 = pd.read_csv('data/borough_race_pop_sqf.csv')
df2 = pd.read_csv('data/borough_pop_sqftotal.csv')

In [4]:
# creating new dataframe with percentages score 
columns = ['borough','race','race_pop','race_pop_per','sqf_count','sqf_count_per']
df = pd.DataFrame(columns=columns)
df['borough'] = df1['borough']
df['race'] = df1 ['race']
df['race_pop'] = df1 ['est_pop']
df['sqf_count'] = df1 ['sqf_count']

for i in range(0,len(df)):
    borough = df['borough'].iloc[i]
    
    for j in range (0,len(df2)):
        if df2['borough'].iloc[j].lower() == borough:
            pop = df2['pop'].iloc[j]
            sqf_total = df2['sqf_total'].iloc[j]
    
    df['race_pop_per'].iloc[i] = round(df['race_pop'].iloc[i]/pop*100,2)
    df['sqf_count_per'].iloc[i] = round(df['sqf_count'].iloc[i]/sqf_total*100,2)  

df.to_excel("boro_pop_sqf_breakdown.xlsx")
    
df


/Users/livernteo/Documents/lvt_projects/env/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,borough,race,race_pop,race_pop_per,sqf_count,sqf_count_per
0,bronx,asian / pacific islander,51399,3.57,12,0.4
1,bronx,black,421275,29.3,1548,51.91
2,bronx,hispanic_latino,803636,55.89,1286,43.13
3,bronx,other,27688,1.93,42,1.41
4,bronx,white,133874,9.31,94,3.15
5,brooklyn,asian / pacific islander,305716,11.75,77,1.54
6,brooklyn,black,787705,30.29,3574,71.52
7,brooklyn,hispanic_latino,499279,19.2,882,17.65
8,brooklyn,other,67288,2.59,47,0.94
9,brooklyn,white,940759,36.17,417,8.35


In [5]:
# 1 in N number of race_citizens can expect to be stopped and frisked
columns= ['borough','race','one_every']
oe = pd.DataFrame(columns=columns)
oe['borough'] = df['borough']
oe['race'] = df ['race']
oe['one_every'] = round(df['race_pop']/df['sqf_count'],0)

oe.to_excel("one_every_boro.xlsx")
oe

,borough,race,one_every
0,bronx,asian / pacific islander,4283.0
1,bronx,black,272.0
2,bronx,hispanic_latino,625.0
3,bronx,other,659.0
4,bronx,white,1424.0
5,brooklyn,asian / pacific islander,3970.0
6,brooklyn,black,220.0
7,brooklyn,hispanic_latino,566.0
8,brooklyn,other,1432.0
9,brooklyn,white,2256.0
